In [1]:
import re
import json
import ast

import pandas as pd
from tqdm import tqdm
import numpy as np

import starrydata as sd

In [2]:
sd_dataset = sd.load_dataset(zip_path="/Users/matotomoya/Downloads/starrydata_dataset (2).zip")

df_sample = pd.read_csv(sd_dataset.samples_csv)
df_paper = pd.read_csv(sd_dataset.papers_csv).reset_index(drop=True)
df = df_sample.merge(df_paper, on="SID")
df


2024-07-02 19:37:57,969 - INFO - Loading ZIP file from local path: /Users/matotomoya/Downloads/starrydata_dataset (2).zip
2024-07-02 19:37:57,982 - INFO - Local ZIP file loaded into memory.


Dataset timestamp: 2024-07-01 12:00:01 UTC+0900 (JST)


,sample_name,sample_id,composition,composition_details,SID,DOI_x,created_at_x,updated_at,sample_info,DOI_y,...,title,container_title,container_title_short,volume,issue,page,ISSN,publisher,project_names,created_at_y
0,Pb1Te1I0.0006,1,Pb1Te1I0.0006,NaN,4419,10.1016/j.physb.2005.03.022,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1016/j.physb.2005.03.022,...,"""High performance n-type PbTe-based materials ...","""Physica B: Condensed Matter""","""Physica B: Condensed Matter""","""363""","""1-4""","""196-205""",0921-4526,Elsevier BV,"[""ThermoelectricMaterials"",""GeneralDB""]",Thu Jan 25 2018 13:57:06 GMT+0900 (JST)
1,Pb1Te0.9945I0.0055,2,Pb1Te0.9945I0.0055,NaN,860,10.1039/c1ee01314a,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),"{""LaserFlash"":{""category"":"""",""comment"":""""},""Fo...",10.1039/c1ee01314a,...,"""Reevaluation of PbTe1−xIx as high performance...","""Energy & Environmental Science""","""Energy Environ. Sci.""","""4""","""6""","""2090""","1754-5692,1754-5706",Royal Society of Chemistry (RSC),"[""ThermoelectricMaterials"",""GeneralDB""]",Thu Jan 25 2018 13:56:57 GMT+0900 (JST)
2,In0.01Ag0.01Sn0.98Te1,3,In0.01Ag0.01Sn0.98Te1,NaN,6963,10.1063/1.4959845,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1063/1.4959845,...,"""Thermoelectric performance of co-doped SnTe w...","""Applied Physics Letters""","""Appl. Phys. Lett.""","""109""","""4""","""042102""","0003-6951,1077-3118",AIP Publishing,"[""ThermoelectricMaterials""]",Thu Jan 25 2018 13:57:14 GMT+0900 (JST)
3,Pb1.001Te1I0.002,4,Pb1.001Te1I0.002,NaN,4419,10.1016/j.physb.2005.03.022,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1016/j.physb.2005.03.022,...,"""High performance n-type PbTe-based materials ...","""Physica B: Condensed Matter""","""Physica B: Condensed Matter""","""363""","""1-4""","""196-205""",0921-4526,Elsevier BV,"[""ThermoelectricMaterials"",""GeneralDB""]",Thu Jan 25 2018 13:57:06 GMT+0900 (JST)
4,Pb1Te0.9988I0.0012,5,Pb1Te0.9988I0.0012,NaN,860,10.1039/c1ee01314a,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),"{""LaserFlash"":{""category"":"""",""comment"":""""},""Fo...",10.1039/c1ee01314a,...,"""Reevaluation of PbTe1−xIx as high performance...","""Energy & Environmental Science""","""Energy Environ. Sci.""","""4""","""6""","""2090""","1754-5692,1754-5706",Royal Society of Chemistry (RSC),"[""ThermoelectricMaterials"",""GeneralDB""]",Thu Jan 25 2018 13:56:57 GMT+0900 (JST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78771,sample2 Hext ⊥ HA,79055,Nd2Fe14B,NaN,48522,10.1016/0304-8853(95)00495-5,Fri Jun 28 2024 16:09:29 GMT+0900 (JST),Fri Jun 28 2024 16:09:29 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1016/0304-8853(95)00495-5,...,"""Exploration of the magnetic field effect on t...","""Journal of Magnetism and Magnetic Materials""","""Journal of Magnetism and Magnetic Materials""",153,1,"""224-230""",0304-8853,Elsevier BV,"[""MagneticMaterials""]",Fri Jun 28 2024 11:48:24 GMT+0900 (JST)
78772,sample2 Hext // HA,79056,Nd2Fe14B,NaN,48522,10.1016/0304-8853(95)00495-5,Fri Jun 28 2024 16:11:44 GMT+0900 (JST),Fri Jun 28 2024 16:11:44 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1016/0304-8853(95)00495-5,...,"""Exploration of the magnetic field effect on t...","""Journal of Magnetism and Magnetic Materials""","""Journal of Magnetism and Magnetic Materials""",153,1,"""224-230""",0304-8853,Elsevier BV,"[""MagneticMaterials""]",Fri Jun 28 2024 11:48:24 GMT+0900 (JST)
78773,no field,79057,Nd2.4Pr5.6DyFe84MoB6,NaN,48524,10.1063/1.1853693,Mon Jul 01 2024 10:35:24 GMT+0900 (JST),Mon Jul 01 2024 10:35:24 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1063/1.1853693,...,"""Magnetization reversal and nanostructure refi...","""Journal of Applie

In [3]:
df['sample_info']

0                                                      NaN
1        {"LaserFlash":{"category":"","comment":""},"Fo...
2                                                      NaN
3                                                      NaN
4        {"LaserFlash":{"category":"","comment":""},"Fo...
                               ...                        
78771    {" remanence magnetion":{"category":"","commen...
78772    {" remanence magnetion":{"category":"","commen...
78773    {" remanence magnetion":{"category":"","commen...
78774    {" remanence magnetion":{"category":"","commen...
78775    {" remanence magnetion":{"category":"","commen...
Name: sample_info, Length: 78776, dtype: object

In [4]:
import json
from tqdm import tqdm

# ユニークなdescriptorを収集するセット
unique_descriptors = set()

# すべての行をスキャンしてユニークなdescriptorを収集
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    try:
        sample_info_json = json.loads(row['sample_info'])
        for descriptor in sample_info_json.keys():
            unique_descriptors.add(descriptor.replace(" ", "_").lower())
    except Exception as error:
        # "JSONファイルの読み込みに失敗しましたが、Descriptorのuniqueリストを作成するだけなので、続行
        continue

unique_descriptors

100%|██████████| 78776/78776 [00:01<00:00, 64808.33it/s]


{'2al,_3al,_4al',
 '2d_iron-tetracyanoquinodimethane',
 '__measurement_temperature',
 '__remanence_magnetion',
 '_coercivity',
 '_measurement_temperature',
 '_remanence_magnetion',
 '_saturation_magnetization',
 'absolutedensity',
 'ac/qc_type',
 'annealing_condition',
 'anode_active_material',
 'anode_additive_1',
 'anode_additive_2',
 'anode_areal_capacity',
 'anode_binder_1',
 'anode_binder_2',
 'anode_conductive_agent',
 'anode_current_collector',
 'applied_field',
 'cathode_active_material',
 'cathode_additive_1',
 'cathode_additive_2',
 'cathode_areal_capacity',
 'cathode_areal_density_(cathode_loading)',
 'cathode_binder_1',
 'cathode_binder_2',
 'cathode_conductive_agent',
 'cathode_current_collector',
 'cathode_volumetric_density',
 'cell_type',
 'closedporosity',
 'coercivity',
 'comment',
 'composition_conversion',
 'composition_type',
 'coulombic_efficiency',
 'counter_electrode',
 'current_density',
 'datatype',
 'differential_thermal_analyzer',
 'differentialthermalanalyz

In [5]:
import re

# 新しい列を格納する辞書
new_columns = {f"{descriptor}_category": [None] * df.shape[0] for descriptor in unique_descriptors}
new_columns.update({f"{descriptor}_comment": [None] * df.shape[0] for descriptor in unique_descriptors})

# すべての行を処理して、新しい列にデータを追加
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    if pd.isna(row['sample_info']):
        continue
    sample_info_json = json.loads(row['sample_info'])
    for descriptor, value_object in sample_info_json.items():
        descriptor = descriptor.replace(" ", "_").lower()  # keyを変数として使用
        if "category" in value_object:
            new_columns[f"{descriptor}_category"][index] = value_object["category"]
        if "comment" in value_object:
            new_columns[f"{descriptor}_comment"][index] = value_object["comment"]

# 新しい列を一度にデータフレームに追加
df_new_columns = pd.DataFrame(new_columns)
df_2 = pd.concat([df, df_new_columns], axis=1)
df_2

100%|██████████| 78776/78776 [00:01<00:00, 51088.56it/s]


,sample_name,sample_id,composition,composition_details,SID,DOI_x,created_at_x,updated_at,sample_info,DOI_y,...,measurement_temperatur_comment,maximum_power_comment,coulombic_efficiency_comment,separator_comment,ratio_of_electrolyte_to_sulfur_(e/s_ratio)_comment,emanence_magnetion_comment,sintering_temperature_comment,solvent_3_comment,lattice_parameter_comment,anode_areal_capacity_comment
0,Pb1Te1I0.0006,1,Pb1Te1I0.0006,NaN,4419,10.1016/j.physb.2005.03.022,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1016/j.physb.2005.03.022,...,None,None,None,None,None,None,None,None,None,None
1,Pb1Te0.9945I0.0055,2,Pb1Te0.9945I0.0055,NaN,860,10.1039/c1ee01314a,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),"{""LaserFlash"":{""category"":"""",""comment"":""""},""Fo...",10.1039/c1ee01314a,...,None,None,None,None,None,None,None,None,None,None
2,In0.01Ag0.01Sn0.98Te1,3,In0.01Ag0.01Sn0.98Te1,NaN,6963,10.1063/1.4959845,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1063/1.4959845,...,None,None,None,None,None,None,None,None,None,None
3,Pb1.001Te1I0.002,4,Pb1.001Te1I0.002,NaN,4419,10.1016/j.physb.2005.03.022,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),NaN,10.1016/j.physb.2005.03.022,...,None,None,None,None,None,None,None,None,None,None
4,Pb1Te0.9988I0.0012,5,Pb1Te0.9988I0.0012,NaN,860,10.1039/c1ee01314a,Fri Sep 01 2017 18:19:55 GMT+0900 (JST),Fri Sep 01 2017 18:19:55 GMT+0900 (JST),"{""LaserFlash"":{""category"":"""",""comment"":""""},""Fo...",10.1039/c1ee01314a,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78771,sample2 Hext ⊥ HA,79055,Nd2Fe14B,NaN,48522,10.1016/0304-8853(95)00495-5,Fri Jun 28 2024 16:09:29 GMT+0900 (JST),Fri Jun 28 2024 16:09:29 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1016/0304-8853(95)00495-5,...,,None,None,None,None,,None,None,None,None
78772,sample2 Hext // HA,79056,Nd2Fe14B,NaN,48522,10.1016/0304-8853(95)00495-5,Fri Jun 28 2024 16:11:44 GMT+0900 (JST),Fri Jun 28 2024 16:11:44 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1016/0304-8853(95)00495-5,...,,None,None,None,None,,None,None,None,None
78773,no field,79057,Nd2.4Pr5.6DyFe84MoB6,NaN,48524,10.1063/1.1853693,Mon Jul 01 2024 10:35:24 GMT+0900 (JST),Mon Jul 01 2024 10:35:24 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1063/1.1853693,...,,None,None,None,None,,None,None,None,None
78774,"1.12T,//",79058,Nd2.4Pr5.6DyFe84MoB6,NaN,48524,10.1063/1.1853693,Mon Jul 01 2024 10:37:16 GMT+0900 (JST),Mon Jul 01 2024 10:37:16 GMT+0900 (JST),"{"" remanence magnetion"":{""category"":"""",""commen...",10.1063/1.1853693,...,,None,None,None,None,,None,None,None,None


In [6]:
df_2['datatype_category'].value_counts()

datatype_category
Experiment     40593
                3840
Reference        562
Calculation      346
Unknown          178
NeedCheck         13
Other             11
Name: count, dtype: int64